# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm import tqdm

import time
import torch
import gc

pd.set_option('display.max_columns', None)

## Daten laden

In [2]:
IS_TRAINING = True
if IS_TRAINING:
    FILE = '../../data/processed/train_series_split_normalized.parquet'
else:
    FILE = '../../data/processed/validation_series_split_normalized.parquet'
series = pd.read_parquet(FILE, columns=['num_series_id'])

## Neue Features erstellen

In [3]:
folder = "train/" if IS_TRAINING else "validation/"
file_path_prefix = "../../data/processed/lag-features/" + folder

ANGLEZ_LAGS_FUTURE = [f"anglez_future-lag_{i}" for i in range(-1, -25, -1)]
ENMO_LAGS_FUTURE = [f"enmo_future-lag_{i}" for i in range(-1, -25, -1)]
ANGLEZ_LAGS_PAST = [f"anglez_past-lag_{i}" for i in range(1, 25)]
ENMO_LAGS_PAST = [f"enmo_past-lag_{i}" for i in range(1, 25)]
LAGS_FUTURE = [*ANGLEZ_LAGS_FUTURE, *ENMO_LAGS_FUTURE]
LAGS_PAST = [*ANGLEZ_LAGS_PAST, *ENMO_LAGS_PAST]
FEATURES = ['anglez', 'enmo', *LAGS_PAST, *LAGS_FUTURE]

LABEL = ['awake']

In [4]:
def save_chunk(num_series_id, chunk):
    X = torch.from_numpy(chunk[FEATURES].astype('float32').to_numpy())
    
    torch.save(X, file_path_prefix + str(num_series_id) + ".pt")

In [5]:
def make_features_chunk(num_series_id):
    df = pd.read_parquet(FILE, filters=[('num_series_id','=',num_series_id)])
          
    for i in range(1, 25):        
        df[f"anglez_past-lag_{i}"] = df["anglez"].shift(i).bfill()
        df[f"enmo_past-lag_{i}"] = df["enmo"].shift(i).bfill()

    for i in range(-1, -25, -1):
        df[f"anglez_future-lag_{i}"] = df["anglez"].shift(i).ffill()
        df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
    
    return df.reset_index(drop=True)

In [6]:
def make_features(series):
    overview_data = []
    
    for num_series_id in tqdm(series.num_series_id.unique()):
        chunk = make_features_chunk(num_series_id)
        save_chunk(num_series_id, chunk)
        
        overview_data.append(
            chunk[['num_series_id', 'step', 'awake']].reset_index().rename(columns={'index':'series_index'}).copy()[['num_series_id', 'step', 'awake', 'series_index']]
        )
        
        del chunk
        gc.collect()
    
    return pd.concat(overview_data).reset_index(drop=True)

In [7]:
start_time = time.time()
overview = make_features(series)
print(f'Feature Engineering took {time.time() - start_time} seconds')

  0%|                                                                                          | 0/215 [00:00<?, ?it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
  0%|▍                                                                                 | 1/215 [00:05<18:14,  5.11s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

  7%|██████                                                                           | 16/215 [01:23<18:12,  5.49s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
  8%|██████▍                                                                          | 17/215 [01:28<18:16,  5.54s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 15%|████████████                                                                     | 32/215 [02:52<15:59,  5.24s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 15%|████████████▍                                                                    | 33/215 [02:57<16:32,  5.45s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 22%|██████████████████                                                               | 48/215 [04:20<15:10,  5.45s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 23%|██████████████████▍                                                              | 49/215 [04:25<15:04,  5.45s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 30%|████████████████████████                                                         | 64/215 [05:51<14:32,  5.78s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 30%|████████████████████████▍                                                        | 65/215 [05:57<14:18,  5.72s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

...

 74%|███████████████████████████████████████████████████████████▌                    | 160/215 [17:38<07:13,  7.89s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 75%|███████████████████████████████████████████████████████████▉                    | 161/215 [17:47<07:15,  8.06s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 82%|█████████████████████████████████████████████████████████████████▍              | 176/215 [19:48<05:10,  7.97s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 82%|█████████████████████████████████████████████████████████████████▊              | 177/215 [19:56<05:06,  8.06s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 89%|███████████████████████████████████████████████████████████████████████▍        | 192/215 [22:04<03:10,  8.28s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 90%|███████████████████████████████████████████████████████████████████████▊        | 193/215 [22:12<02:57,  8.05s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 97%|█████████████████████████████████████████████████████████████████████████████▍  | 208/215 [23:13<00:12,  1.84s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 97%|█████████████████████████████████████████████████████████████████████████████▊  | 209/215 [23:15<00:11,  1.96s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_18212\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

Feature Engineering took 1408.1001982688904 seconds


## Overview speichern

In [8]:
overview

,num_series_id,step,awake,series_index
0,1,0,1,0
1,1,1,1,1
2,1,2,1,2
3,1,3,1,3
4,1,4,1,4
...,...,...,...,...
73060068,277,589675,1,589675
73060069,277,589676,1,589676
73060070,277,589677,1,589677
73060071,277,589678,1,589678


In [9]:
overview.to_parquet(file_path_prefix + "overview.parquet")